# ML Estimation Noisy Observations Bounded Confidence Model
In this notebook we estimate the parameter $\epsilon$ in a BC model with noisy observations on the opinions.
Instead of knowing $X$, we observe only $Y$, that is a Bernoulli sample of a sample of agents at each timestep.
$Y_j = 1$ with probability $X_j$.

The likelihood to be optimized is 

$\mathcal{L}(\epsilon) = \sum\limits_{j = 1,\ldots,M} \log ( s^j \kappa_{\epsilon} (e^j) + (1 - s^j) (1 - \kappa_\epsilon(e_t)) )$ $+\sum\limits_{j = 1,\ldots,K} \log \left( y_j f_t(X_0)_j + (1 - y_j) (1 - f_t(X_0)_j) \right)$,

where $\kappa = \sigma(\rho \cdot (\epsilon - | \Delta X |))$ is the probability of having a positive interaction, and $s = 1$ if the interaction is positive, and $s = 0$ otherwise.

$f_t(X_0)$ are the opinions of the agents at time $t$, given the initial opinion $X_0$.

Hence, the loss is composed of two pieces, the neg-log-likelihood of the interactions (that depend on the estimates of the opinions), and the neg-log-likelihood of the opinions from the evidences (Y).

We maximise $\mathcal{L}$ with gradient descent.

In this case, we estimate both $\epsilon$ and $X_0$.

In [1]:
import torch
import numpy as np
import torch.nn as nn

from scipy.special import expit as sigmoid
from scipy.special import logit
from time import time
from tqdm import tqdm

import sys
sys.path += ["../src"]
from simulator_opinion_dynamics import kappa_from_epsilon
import simulator_opinion_dynamics as sod
from initialize_model import EarlyStopping,RandomizeEpsilon,choose_optimizer


In [2]:
# we define a new function for setting up the optimizer, in order to have different learning rates for epsilon and X0
def choose_optimizer_evidences(optimizer_name, lr, model, X0_lr_scale = 1):
    optimizer_list = ["adam", "SGD", "nadam", "adagrad", "RMSprop"]
    assert optimizer_name in optimizer_list, f"Optimizer must be in {optimizer_list}"
    
    if optimizer_name == "adam":
        optimizer = torch.optim.Adam([
            {'params': model.logit_X0, 'lr': lr * X0_lr_scale},
            {'params': model.theta, 'lr': lr}
        ], lr = lr)
        
        optimizer = torch.optim.Adam([
            {'params': model.logit_X0, 'lr': lr * X0_lr_scale},
            {'params': model.theta, 'lr': lr}
        ], lr = lr) #define the optimizer with the input learning rate
    if optimizer_name == "SGD":
        optimizer = torch.optim.SGD([
            {'params': model.logit_X0, 'lr': lr * X0_lr_scale},
            {'params': model.theta, 'lr': lr}
        ], lr = lr, momentum = 0.9) #define the optimizer with the input learning rate
    if optimizer_name == "nadam":
        optimizer = torch.optim.NAdam([
            {'params': model.logit_X0, 'lr': lr * X0_lr_scale},
            {'params': model.theta, 'lr': lr}
        ], lr = lr) #define the optimizer with the input learning rate        
    if optimizer_name == "adagrad":
        optimizer = torch.optim.Adagrad([
            {'params': model.logit_X0, 'lr': lr * X0_lr_scale},
            {'params': model.theta, 'lr': lr}
        ], lr = lr) #define the optimizer with the input learning rate
    if optimizer_name == "RMSprop":
        optimizer = torch.optim.RMSprop([
            {'params': model.logit_X0, 'lr': lr * X0_lr_scale},
            {'params': model.theta, 'lr': lr}
        ], lr = lr)
    return optimizer



# function used for counting the repeated interactions at the same timestep
def count_s_from_edge(e):
    e_unique = e.unique(dim = 0, return_counts = True)
    e_unique[0][:,2] = e_unique[0][:,2] * e_unique[1]
    e_sum_s = e_unique[0]
    return e_sum_s[e_sum_s[:,2] > 0]

# define a sparse tensor to store the adj matrix for each timestep
# this is useful to fast update the opinions at the varying of X0
def edges_coo_mu(edges, mu, N):
        edges_count_s = [count_s_from_edge(edges[t]) for t in range(len(edges))]
        
        M = [torch.sparse_coo_tensor(indices = edges_count_s[t][:,:2].T, dtype = torch.float32,
                                     values = mu * edges_count_s[t][:,2], 
                                     size = [N, N]) for t in range(len(edges))]
        
        return M
    
# fast (deterministic) computation of the tensor X from edges and X0
def X_from_X0_coo_edges(X0, M, T, N):
    X = torch.zeros([T, N], dtype = torch.float32)
    
    X[0] = X0
    for t in range(T - 1):
        updates = ((X[t] * M[t].to_dense()).sum(dim = 1) - (X[t] * M[t].to_dense().T).sum(dim = 0) +\
                   (X[t] * M[t].to_dense().T).sum(dim = 1) - (X[t] * M[t].to_dense()).sum(dim = 0))
        X[t+1] = X[t] + updates
    
    return X
  
# fast computation of diff_X and extraction of X of the nodes with observed evidences
def evidences_diff_from_X0_coo_edges(X0, indices_M, values_M, T, N, edges, evidences_indices):
    diff_X = torch.empty(T-1, len(edges[0]))
    X_ = X0.clone().detach()
    X_evidences = torch.empty(T, len(evidences_indices[0]))
    
    for t in range(T - 1):
        diff_Xt = torch.sparse_coo_tensor(indices = indices_M[t],
                                          values = values_M[t] * (X_[indices_M[t][0]] - X_[indices_M[t][1]]),
                                          size = [N,N])
        
        updates = - torch.sparse.mm(diff_Xt, torch.ones(N,1))[:,0] + torch.sparse.mm(diff_Xt.T, torch.ones(N,1))[:,0]
        
        u,v,_ = edges[t].T
        
        diff_X[t] = X_[u] - X_[v]
        X_evidences[t] = X_[evidences_indices[t]]
        
        X_ += updates
    X_evidences[T-1] = X_[evidences_indices[T-1]]
    
    return diff_X, X_evidences


In [3]:
class BC_evidence_X_Estimation(nn.Module):
    
    def __init__(self, parameters0, X, edges, evidences):
        
        super().__init__()
        
        epsilon0, mu, rho = parameters0
        self.rho = rho
        self.mu = mu
        
        self.u,self.v,self.s,self.t = uvst = sod.convert_edges_uvst(edges)

        self.X = X
        self.diff_X = X[self.t,self.u] - X[self.t,self.v]
        _, self.edge_per_t, _ = edges.size()
        self.T, self.N = X.size()
        
        self.M = edges_coo_mu(edges, mu, self.N)
                
        self.evidences_per_t = len(evidences[0][0])

        self.evidences_indices = torch.cat([evidences[k][0][None,:] for k in range(len(evidences))], dim = 0) #tensor with the indices of the users of which we know the evidence
        self.evidences_opinions = torch.cat([evidences[k][1][None,:] for k in range(len(evidences))], dim = 0).reshape(self.T * self.evidences_per_t).to(torch.float32) #only the evidences of these users
        self.evideneces = evidences
        
        X0 = torch.rand(self.N, dtype = torch.float32, requires_grad = True) # random initialization of the opinions
        self.logit_X0 = nn.Parameter(torch.logit(X0))   #define the parameters of the model
        
        theta = torch.tensor([logit(2 * epsilon0)], requires_grad = True)
        self.theta = nn.Parameter(theta)
        
        
    def forward(self):
        epsilon = torch.sigmoid(self.theta) / 2
        X0 = torch.sigmoid(self.logit_X0)   #at each step clip X0 in the interval [0,1]
        X = X_from_X0_coo_edges(X0, self.M, self.T, self.N)
        
        diff_X = X[self.t,self.u] - X[self.t, self.v] 
        kappa = kappa_from_epsilon(epsilon, diff_X, self.rho) # compute probability of interaction with current estimate of epsilon
        
        return X, kappa
    
    
    def neg_log_likelihood_function(self, kappa, s, evidences_indices, evidences_opinions, evidences_per_t, X, t_minibatch):
        T, _ = X.shape
        real_opinions = torch.cat([X[t, evidences_indices[t]] for t in range(T)])
        
        loss_edges = -torch.sum(torch.log(s * kappa + (1 - s) * (1 - kappa)))
        
        # this steps avoid to obtain an estimate of X0 that is ~1-X0
        # this can happen because the loss of the edges depends only on the distance between X, not on the abs values
        loss_evidences = -torch.sum(torch.log(evidences_opinions * real_opinions + (1 - evidences_opinions) * (1 - real_opinions)))
        loss_evidences_flip = -torch.sum(torch.log(evidences_opinions * (1 - real_opinions) + (1 - evidences_opinions) * (1 - (1 - real_opinions))))
        
        X0_flip = True if loss_evidences > loss_evidences_flip else False
        
        return torch.min(loss_edges + loss_evidences, loss_edges + loss_evidences_flip), X0_flip
    
    def neg_log_likelihood_function_minibatch(self, kappa, s, evidences_indices, evidences_opinions, evidences_per_t, X, t_minibatch):
        T, _ = X.shape
        real_opinions = torch.cat([X[t, evidences_indices[t]] for t in range(T)])
        
        loss_edges = -torch.sum(torch.log(s * kappa + (1 - s) * (1 - kappa))[t_minibatch])
        
        loss_evidences = -torch.sum(torch.log(evidences_opinions * real_opinions + (1 - evidences_opinions) * (1 - real_opinions))[t_minibatch])
        loss_evidences_flip = -torch.sum(torch.log(evidences_opinions * (1 - real_opinions) + (1 - evidences_opinions) * (1 - (1 - real_opinions)))[t_minibatch])
        
        X0_flip = True if loss_evidences > loss_evidences_flip else False
        
        return torch.min(loss_edges + loss_evidences, loss_edges + loss_evidences_flip), X0_flip
    


In [4]:
def gradient_descent_BC_evidence_X(X, edges, evidences, mu, rho, num_epochs,
                                   epsilon0 = 0.25, optimizer_name = "adam",
                                   lr = 0.05,  X0_lr_scale = 1,
                                   hide_progress = True, minibatch_size = 0, seed = None,
                                   early_stopping_kw = {"patience": 20, "min_delta": 1e-5, "min_epochs": 20, "long_run_delta": 1e-5, "long_run_diff":10, "long_run_patience": 5}):
    if seed is not None:
        np.random.seed(seed)
    
    u,v,s,t = uvst = sod.convert_edges_uvst(edges)
    T,_ = X.shape
    
    
    model_class = BC_evidence_X_Estimation
    model = model_class((epsilon0, mu, rho), X, edges, evidences)
    if minibatch_size == 0:
        loss_function = model_class.neg_log_likelihood_function
    if minibatch_size > 0:
        loss_function = model_class.neg_log_likelihood_function_minibatch
    
    early_stopping = EarlyStopping(**early_stopping_kw)
    
    optimizer = choose_optimizer_evidences(optimizer_name, lr, model, X0_lr_scale)
    
    history = {"epsilon": [epsilon0], "loss": [], "X0": []}
    
    t0 = time()
    for epoch in tqdm(range(num_epochs), disable = hide_progress):
        t_minibatch = torch.randperm(T-1)[:minibatch_size]
        
        X_, kappa = model()
        loss, X0_flip = loss_function(model, kappa, model.s, model.evidences_indices, model.evidences_opinions, 
                             model.evidences_per_t, X_, t_minibatch)
        
        loss.backward()
        optimizer.step()
        
        X0_estimate = sigmoid(model.logit_X0.detach()) + (1 - 2 * sigmoid(model.logit_X0.detach())) * X0_flip
        history["X0"].append(X0_estimate)
        history["epsilon"].append(sigmoid(model.theta.item()) / 2)
        history["loss"].append(loss.item())
        
        if loss.item() == np.inf:
            
            model = model_class((np.random.rand() / 2, mu, rho), X, edges, evidences)
            optimizer = choose_optimizer_evidences(optimizer_name, lr, model, X0_lr_scale / 2)

        
        optimizer.zero_grad()
        
        if epoch > early_stopping_kw["min_epochs"]:
            
            early_stopping(history["epsilon"][-2], history["epsilon"][-1], history["epsilon"][-early_stopping_kw["long_run_diff"]], epoch)
            if early_stopping.early_stop:
                break
    
    
    t1 = time()
    history["time"] = t1 - t0
    
    return history



In [7]:
N, T, edge_per_t = 100, 256, 4
evidences_per_t = 4
epsilon, mu, rho = 0.35, 0.4, 16

X, edges, evidences = sod.simulate_BC(N, T, edge_per_t, evidences_per_t, (epsilon, mu, rho), seed = 34945)

In [16]:
history = gradient_descent_BC_evidence_X(X, edges, evidences, num_epochs = 200, 
                                         epsilon0 = 0.25, mu = mu, rho = rho, 
                                         optimizer_name = "RMSprop",
                                         lr = 0.05, X0_lr_scale = 10, seed = 91368)
    